### Day 17, Part 2: Actual


#### Details:

- We shift from 3D to 4D

In [1]:
filepath = "day17_data.txt"
with open(filepath) as fh:
    lines = [line.strip() for line in fh.readlines()]

In [2]:
# get it into a matrix 
import numpy as np

matrix_in = [list(code) for code in lines]

# matrix form of z = 0 -> we assume the surroundings are all inactive at the start
matrix = np.array([np.array(row) for row in matrix_in])

# going to switch to 1s and 0s
# 1 == Active, 0 == Inactive -> will let me sum
matrix[matrix == '#'] = 1
matrix[matrix == '.'] = 0
z0 = matrix.astype(int)
print(z0)

[[1 1 1 1 1 1 0 1]
 [1 1 0 1 1 1 0 1]
 [1 0 1 1 1 0 1 1]
 [0 0 1 0 0 1 1 1]
 [1 1 0 1 0 1 0 1]
 [1 1 0 0 0 1 1 0]
 [1 0 1 0 1 1 0 1]
 [0 1 1 1 0 1 1 1]]


In [3]:
# we now need a way to expand these outward in all directions.
# initially we have all 0s for z = -1 & z = 1 
# if something does not exist, we can just assume it is 0
z1 = np.full(z0.shape, 0)
zn1 = np.full(z0.shape, 0)

In [5]:
array3d = np.stack((z1, z0, zn1))
array0s = np.stack((z1, z1, zn1)) # want to ensure I am building a 3D set of 0s

In [6]:
# 4d array can just do another stack combination
# np.stack is going to increase dimension of all inputs by 1
array4d = np.stack((array3d, array0s, array0s))
print(array4d.shape)

(3, 3, 8, 8)


In [7]:
# single func
def countNeighbors(coord, matrix):
    """Review neighbor states"""
    depth, row, col, hyp = coord
    count = 0
    tot_neigh = 0
    for d,r,c,h in [(depth + i, row + j, col + k, hyp + l) for i in (-1,0,1) for j in (-1,0,1) for k in (-1,0,1) for l in (-1,0,1)]:
        if d == depth and r == row and c == col and h == hyp:
            continue
        tot_neigh += 1
        try:
            count += matrix[(d,r,c,h)]
        except:
            pass

    return count, tot_neigh
    

In [8]:
countNeighbors((1,1,1,1), array4d)

(7, 80)

### Run 6 Iterations:

- Going to expand the padding each step by 1 to account for the inifinite expansion

In [9]:
# Just going to pad in each loop
final_matrix = np.pad(array4d, pad_width=1, mode='constant', constant_values=0)
print(f"New shape is: {final_matrix.shape}")

for _ in range(6):

    # array copy, but all 0s:
    my_copy = np.empty_like(final_matrix)

    # iterate over each value
    for depth in range(final_matrix.shape[0]):
        for row in range(final_matrix.shape[1]):
            for col in range(final_matrix.shape[2]):
                for hyp in range(final_matrix.shape[3]):

                        # find state & count neighbor states
                        coord = (depth, row, col, hyp)
                        state = final_matrix[coord]
                        active_count, tot_n = countNeighbors(coord, final_matrix)

                        if state:
                            # keep active
                            if 2 <= active_count <= 3:
                                my_copy[coord] = 1
                            else:
                                my_copy[coord] = 0
                        else:
                            if active_count == 3:
                                my_copy[coord] = 1
                            else:
                                my_copy[coord] = 0

    # update
    final_matrix = np.array(my_copy, copy=True)
    
    # Add padding 
    final_matrix = np.pad(final_matrix, pad_width=1, mode='constant', constant_values=0)
    print(f"New shape is: {final_matrix.shape}")
    
# print active:
print(f"Total active: {np.sum(final_matrix)}")

New shape is: (5, 5, 10, 10)
New shape is: (7, 7, 12, 12)
New shape is: (9, 9, 14, 14)
New shape is: (11, 11, 16, 16)
New shape is: (13, 13, 18, 18)
New shape is: (15, 15, 20, 20)
New shape is: (17, 17, 22, 22)
Total active: 2236
